### twine

transform harlowe passages into 

function that takes current passage and story variables and returns
- node appropriate for consumption by display
    - list of linked passages is included

### need rules for different passage tags

- **map**
    - `$mapImage` becomes backdrop, defaults to oval office
    - `$places` becomes places, defaults to 'abcd...' in a circle
        - alternative: highlight objects or outlines with same-size transparent png

- **encounter**
    - enemy determined by text with `<enemy|` tag
        - footer checks this against existing enemies, should generate with cmpd.web
        - **rather than the following crap, define the enemies entirely in python for now**
    - image with `<enemyImage|` nametag can override image in stable
        - extract URL
    - override other properties as needed
        - `<enemyClass|`
        - `<**kwargs|` to enemyClass init

- **message**
    - separate passage, but can overlay on previous if `overlay` tag present
    - links are choices, can re-use map-menu Component or rewrite to map keys to class="select"
    - fancy stuff later
    

- **all passages**    
    - transitions ( `HarloweLink` ) made with `(link: )(goto: )` syntax can contain `(set: )` macros
    - after conditions are applied links with <instant| tag trigger an immediate transition from the GameMaster

- **walkabout**
    - why not

In [187]:
grammar = {
    'S': ('A N',),
    'A': ('Constipated', 'Disjointed'),
    'N': ('Garbageman', 'Hack', 'A N')
}

In [196]:
from cmpd_web import load_sheet
import pandas as pd
import os

In [197]:
os.environ['KEY1']='31f1300f-e255-41'
os.environ['KEY2']='11664c86-ad0f-44'


In [254]:
zip(*[list('ab'), list('cd')])

[('a', 'c'), ('b', 'd')]

In [247]:
def load_remote(sheet):

    sheet = load_sheet(sheet)
    sheet = [[x.strip() for x in y] for y in sheet]
    df = pd.DataFrame(sheet[1:], columns=sheet[0])
    vocab, subsets = [], []
    for col in df:
        if col:
            words = list(df[col])
            vocab += [(col, words)]
    return df

df = load_remote('DIDB')

In [248]:
def subset(df, col='_start'):
    df2 = df[df[col] != '']
    d = {}
    for symbol, df3 in df2.groupby('symbol'):
        d[symbol] = tuple(df3['symbol chain'])
    return d

grammar = subset(df)

In [249]:
grammar

{'A': ('Candy-Assed', 'Constipated', 'Tufted', 'Clamorous', 'Despicable'),
 'Group': ('a Family of NPL', 'a Syndicate of NPL', 'a Junta of NPL'),
 'N': ('A N', 'Creature', 'Weasel', 'Mouth-Breather', 'Racist', 'Yokel'),
 'NPL': ('A NPL', 'Dwarves', 'Liars', 'Rapists'),
 'PP': ('from Place', 'born into Group', 'employed by Group', 'for Group'),
 'Place': ('the Back of Beyond', 'the Hollow Earth', 'a One-Horse Town'),
 'S': ('N', 'N PP')}

In [256]:
None is not None

False

In [235]:
def is_complete(grammar, so_far):
    return so_far in constructs(grammar, so_far)

is_complete(grammar, 'Clamorous'.split())

False

In [255]:
constructs(grammar, 'Clamorous Racist employed by a Family of Despicable'.split())

[['Clamorous', 'Racist'],
 ['Clamorous',
  'Racist',
  'employed',
  'by',
  'a',
  'Family',
  'of',
  'Despicable',
  'Candy-Assed',
  'NPL'],
 ['Clamorous',
  'Racist',
  'employed',
  'by',
  'a',
  'Family',
  'of',
  'Despicable',
  'Constipated',
  'NPL'],
 ['Clamorous',
  'Racist',
  'employed',
  'by',
  'a',
  'Family',
  'of',
  'Despicable',
  'Tufted',
  'NPL'],
 ['Clamorous',
  'Racist',
  'employed',
  'by',
  'a',
  'Family',
  'of',
  'Despicable',
  'Clamorous',
  'NPL'],
 ['Clamorous',
  'Racist',
  'employed',
  'by',
  'a',
  'Family',
  'of',
  'Despicable',
  'Despicable',
  'NPL'],
 ['Clamorous',
  'Racist',
  'employed',
  'by',
  'a',
  'Family',
  'of',
  'Despicable',
  'Dwarves'],
 ['Clamorous',
  'Racist',
  'employed',
  'by',
  'a',
  'Family',
  'of',
  'Despicable',
  'Liars'],
 ['Clamorous',
  'Racist',
  'employed',
  'by',
  'a',
  'Family',
  'of',
  'Despicable',
  'Rapists']]

In [184]:
def load_vocab(sheet, reload=False, subset=None):
    """ Load columns of google sheet into vocab of type [(header, [word])]. 
    No entry for empty header. Empty and duplicate words removed.

    Uses local cached json if reload is False and file exists.
    """
    path = 'resources/vocab/%s.json' % sheet



    def load_remote(sheet):

        sheet = load_sheet(sheet)
        df = pd.DataFrame(sheet[1:], columns=sheet[0])
        vocab, subsets = [], []
        for col in df:
            if col:
                words = list(df[col])
                vocab += [(col, words)]

        with open(path, 'w') as fh:
            json.dump(vocab, fh, indent=4)
        return vocab

    def filter_vocab(vocab, subset):
        """ If subset is None, all words returned.
        """
        if subset is None:
            subset = ''
        actual_vocab = []
        subset_index = []
        for (col, words) in vocab:
            if col.startswith('_') and re.findall(subset, col):
                subset_index += [i for i,x in enumerate(words) if x]
        subset_index = sorted(set(subset_index))
        
        for (col, words) in vocab:
            if not col.startswith('_'):
                if subset:
                    words = [w for i, w in enumerate(words) 
                               if i in subset_index]
                words = sorted(set(words))
                actual_vocab += [(col, words)]

        return actual_vocab


    if reload:
        vocab = load_remote(sheet)
    else:
        try:
            with open(path, 'r') as fh:
                vocab = json.load(fh)
        except (IOError, ValueError):
            vocab = load_remote(sheet)

    return filter_vocab(vocab, subset)


In [189]:
choices(grammar, 'Constipated'.split())

['Constipated', 'Disjointed', 'Garbageman', 'Hack']

In [144]:
grammar = {
    'S': ['NounPhrase', 'NounPhrase RelativeClause'],
    'NounPhrase': ['Noun', 'Adjective Noun'],
    'Noun': ['Garbageman', 'Hack'],
    'Adjective': ['Constipated', 'Disjointed'],
    'RelativeClause': ['from Place'],
    'Place': ['the Back of Beyond', 'the Hollow Earth', 'a One-Horse Town']
}

In [86]:
# depth-first
# Grammar -> Sentence -> List Sentence
def extend(grammar, sentence, break_on=lambda x: False, discard=lambda x: False):
    kwargs = {'break_on': break_on, 'discard': discard}
    if discard(sentence):
        return []
    if break_on(sentence):
        return [sentence]
    
    result = []
    for i, symbol in enumerate(sentence):
        if symbol in grammar:
            for symbol_chain in grammar[symbol]:
                new_sentence = sentence[:i] + symbol_chain.split() + sentence[i+1:]
                result += extend(grammar, new_sentence, **kwargs)
            return result
    return [sentence]

[' '.join(x) for x in extend(grammar, ['S'])]

['Garbageman',
 'Hack',
 'Constipated Garbageman',
 'Constipated Hack',
 'Disjointed Garbageman',
 'Disjointed Hack',
 'Garbageman from the Back of Beyond',
 'Garbageman from the Hollow Earth',
 'Garbageman from a One-Horse Town',
 'Hack from the Back of Beyond',
 'Hack from the Hollow Earth',
 'Hack from a One-Horse Town',
 'Constipated Garbageman from the Back of Beyond',
 'Constipated Garbageman from the Hollow Earth',
 'Constipated Garbageman from a One-Horse Town',
 'Constipated Hack from the Back of Beyond',
 'Constipated Hack from the Hollow Earth',
 'Constipated Hack from a One-Horse Town',
 'Disjointed Garbageman from the Back of Beyond',
 'Disjointed Garbageman from the Hollow Earth',
 'Disjointed Garbageman from a One-Horse Town',
 'Disjointed Hack from the Back of Beyond',
 'Disjointed Hack from the Hollow Earth',
 'Disjointed Hack from a One-Horse Town']

In [85]:
# breadth-first
# Grammar -> List Sentence -> List Sentence
def extend(grammar, sentences):
    for j, head in enumerate(sentences):
        head = sentences[j]
        tail = sentences[j+1:] + sentences[:j]

        for i, symbol in enumerate(head):
            if symbol in grammar:
                for symbol_chain in grammar[symbol]:
                    new_sentence = head[:i] + symbol_chain.split() + head[i+1:]
                    tail.append(new_sentence)
                return extend(grammar, tail)
    return sentences
    

[' '.join(x) for x in extend(grammar,[['S']])]

['Garbageman',
 'Hack',
 'Constipated Garbageman',
 'Constipated Hack',
 'Disjointed Garbageman',
 'Disjointed Hack',
 'Garbageman from the Back of Beyond',
 'Garbageman from the Hollow Earth',
 'Garbageman from a One-Horse Town',
 'Hack from the Back of Beyond',
 'Hack from the Hollow Earth',
 'Hack from a One-Horse Town',
 'Constipated Garbageman from the Back of Beyond',
 'Constipated Garbageman from the Hollow Earth',
 'Constipated Garbageman from a One-Horse Town',
 'Constipated Hack from the Back of Beyond',
 'Constipated Hack from the Hollow Earth',
 'Constipated Hack from a One-Horse Town',
 'Disjointed Garbageman from the Back of Beyond',
 'Disjointed Garbageman from the Hollow Earth',
 'Disjointed Garbageman from a One-Horse Town',
 'Disjointed Hack from the Back of Beyond',
 'Disjointed Hack from the Hollow Earth',
 'Disjointed Hack from a One-Horse Town']

In [167]:
import external.cfg
reload(external.cfg)
from external.cfg import choices, constructs

In [165]:
choices(grammar, 'Disjointed Garbageman from the Back of Beyond'.split())

[]

In [172]:
constructs(grammar, 'idiot'.split())

[]

In [170]:
constructs(grammar, 'Disjointed Garbageman'.split())

[['Disjointed', 'Garbageman'], ['Disjointed', 'Garbageman', 'from', 'Place']]

In [166]:
choices(grammar, 'idiot')

[]

In [23]:
from external.inspect import inspect
from glob import glob

srcs = glob('elm-src/*.elm') + glob('elm-src/*/*.elm')
inspect(srcs)

,elm file,python,definition
type alias,,,
AttributeList,elm-src/Bar.elm,??,List (Html.Attribute Msg)
Choice,elm-src/Message/Message.elm,cmpd_web.Choice,"{ label : String, key : String, name : String }"
Enemy,elm-src/Versus/Types.elm,cmpd_web.Enemy,"{ image : String, name : String, health : Float }"
Map,elm-src/Map/Types.elm,cmpd_web.Map,"{ image : String, name : String, places : List..."
Message,elm-src/Message/Message.elm,cmpd_web.Message,"{ text : String, choices : List Choice, name :..."
Place,elm-src/Game/Types.elm,cmpd_web.Place,"{ x : Float, y : Float, label : String, key : ..."
Place,elm-src/Map/Types.elm,cmpd_web.Place,"{ x : Float, y : Float, label : String, key : ..."
Place_,elm-src/Game/Types.elm,??,"{ x : Float, y : Float, label : String, key : ..."
Player,elm-src/Versus/Types.elm,??,"{ loaded : List Word, unloaded : List Word, ca..."


In [1]:
from glob import glob
import re
from collections import namedtuple
import sys
sys.path.append('/Users/feldman/packages/harlowe-processor/')

In [79]:
passage_filter = 'Dummy office', 'Challenge ctenophora', 'Images'

In [88]:
dummy_office = [m for m in maps if m.name == 'Dummy office'][0]
challenge_c =  encounters['Challenge ctenophora']

Gamemaster sets up the permitted transitions
transition is called, don't use previous state if possible
Gamemaster has method to update elm based on node type
    - send encounter
    - send map
    - send message

<module 'external.harlowe_extra' from 'external/harlowe_extra.py'>

In [266]:
reload(external.harlowe_extra)
from external.harlowe_extra import html_to_nodes
reload(cmpd_web)
from cmpd_web import HarloweLocal
vocab = cmpd_web.load_vocab('derp')
grammar = None
player = cmpd_web.Player(vocab, grammar)

In [327]:
reload(cmpd_web)
from cmpd_web import GameMaster, Player, LocalGameMaster
vocab = cmpd_web.load_vocab('derp')
player = Player(vocab, grammar)
GM = LocalGameMaster([], 'Dummy office', player)
GM.load_html(html)

GM.initialize()

map with 4 places in Gallery of Heroes
map with 7 places in Oval Office
map with 1 places in Dummy office
generalAbrams found in Fight general
ctenophora found in Fight ctenophora
ctenophora found in Challenge ctenophora
image: https://s4.postimg.io/3otw0adgd/ovaloffice.png
places: [Place(x=0.24, y=0.77, key='c', label='Challenge ctenophora')]
Current map: Dummy office
Go to
Challenge ctenophora



In [340]:
m = GM.nodes['Dummy office']

In [345]:
hasattr([], '_asdict')

False

In [347]:
def asdict(d):
    try:
        return d._asdict()
    except AttributeError:
        return d

In [348]:
m._asdict()['places']

[Place(x=0.24, y=0.77, key='c', label='Challenge ctenophora')]

In [313]:
GM.transition('Challenge ctenophora')

encounter with {'image': 'images/ctenophora.png', 'health': 1, 'name': 'ctenophora'}
Wordbank: Lazy, Stupid, Insecure, Idiotic, ...
Enemy: Enemy(name='ctenophora', image='images/ctenophora.png', cls=<class 'cmpd_web.Opponent'>, vocab='more')
Enter text to insult, enter nothing to quit.
vapid
Wordbank: Douche, Ass, Turd, Butt, ...
butt
Wordbank: Pilot, Captain, Pirate, Knob, ...
pilot
#################################################
# [--------------------------                 ] #
# Stupid Butt Pilot (0.10)                      #
#                               damaged fascist #
#################################################
Wordbank: Lazy, Stupid, Insecure, Idiotic, ...



In [334]:
maps

{'Dummy office': Map(name='Dummy office', image='https://s4.postimg.io/3otw0adgd/ovaloffice.png', places=[Place(x=0.24, y=0.77, key='c', label='Challenge ctenophora')]),
 'Gallery of Heroes': Map(name='Gallery of Heroes', image='http://online.rapidresizer.com/patternizeimg.php?builtin=logo&id=4cc46cdf56dbdec3f2cde2e1b78f6c5d&bw=1&edge=1&threshold=64&w=', places=[Place(x=0.24, y=0.77, key='n', label=u'RMN'), Place(x=0.06, y=0.46, key='e', label=u'Ehrlichmann'), Place(x=0.27, y=0.42, key='c', label=u'Colson'), Place(x=0.72, y=0.27, key='h', label=u'Haldeman')]),
 'Oval Office': Map(name='Oval Office', image='https://s4.postimg.io/3otw0adgd/ovaloffice.png', places=[Place(x=0.24, y=0.77, key='a', label='War room'), Place(x=0.06, y=0.46, key='v', label="President's phone"), Place(x=0.27, y=0.42, key='i', label='Presidential rocking chair'), Place(x=0.72, y=0.27, key='g', label='Cowboy statue'), Place(x=0.61, y=0.22, key='s', label='Flag of Nixon'), Place(x=0.5, y=0.9, key='d', label='RMN'),

In [323]:
p = passages['Dummy office']
p.contents

"An office for losers.\n=><=\n[]<map|\n(set: $mapImage to $ovalOfficeImage)\n<=\n\n[[Challenge the beast->Challenge ctenophora]]\n\n\n(set: $places to \n\t(a:\n\t\t(a: 0.24, 0.77, 'c'),\n\t\t(a: 0.06, 0.46, 'v'),\n\t\t(a: 0.27, 0.42, 'i'),\n\t\t(a: 0.72, 0.27, 'g'),\n\t\t(a: 0.61, 0.22, 's'),\n\t\t(a: 0.50, 0.90, 'd'),\n\t\t\n\t)\n)\t\t \n\t\t\t "

### modify namedtuple to transmit as dict...

In [152]:
reload(cmpd_web)
from cmpd_web import stable

import external.harlowe_extra
reload(external.harlowe_extra)
from external.harlowe_extra import (get_set_macros, 
                                    parse_harlowe_html, 
                                    get_set_args,
                                    get_image_urls,
                                    get_named_hooks,
                                    find_map,
                                    find_encounter,
                                    find_links)

htmls = glob('/Users/feldman/Downloads/test*.html')
html = htmls[0]
print html

a, b, passages = parse_harlowe_html(html)

# find global image variables
image_passages = [p for n,p in passages.items() if 'Images' in n]
image_urls = {}
[image_urls.update(get_image_urls(p)) for p in image_passages]


def build_graph(html):
    a, b, passages = parse_harlowe_html(html)
    
    # find global image variables
    image_passages = [p for n,p in passages.items() if 'Images' in n]
    image_urls = {}
    [image_urls.update(get_image_urls(p)) for p in image_passages]
    
    filter_out = 'header', 'footer', 'startup'
    remaining_passages = [p for p in passages.values()
                             if not any(f in p.tags for f in filter_out)]
    
    # find maps
    maps = [find_map(p, image_urls) for p in remaining_passages]
    maps = {m.name: m for m in maps if m}
    
    # find encounters
    remaining_passages = [p for p in passages.values() 
                              if p.name not in maps]
    encounters = {p.name: find_encounter(p) for p in remaining_passages}
    encounters = {k: v for k,v in encounters.items() if v}
    
    # check graph integrity
    # missing links, ...
    
    return maps, encounters


maps, encounters = build_graph(html)
maps, encounters

/Users/feldman/Downloads/test.html
map with 4 places in Gallery of Heroes
no <map| nametag found in RMN
no <map| nametag found in Ehrlichmann
no <map| nametag found in Colson
no <map| nametag found in Haldeman
no <map| nametag found in Hall of Champions
map with 7 places in Oval Office
no <map| nametag found in President's phone
no <map| nametag found in Cowboy statue
no <map| nametag found in Presidential rocking chair
no <map| nametag found in Flag of Nixon
no <map| nametag found in Fight general
no <map| nametag found in War room
no <map| nametag found in Fight ctenophora
no <map| nametag found in Spiro Agnew
map with 1 places in Dummy office
no <map| nametag found in Challenge ctenophora
no <enemy| nametag found in RMN
no <enemy| nametag found in Images
no <enemy| nametag found in Ehrlichmann
no <enemy| nametag found in Colson
no <enemy| nametag found in Haldeman
no <enemy| nametag found in Hall of Champions
no <enemy| nametag found in President's phone
no <enemy| nametag found in 

({'Dummy office': Map(name='Dummy office', image='https://s4.postimg.io/3otw0adgd/ovaloffice.png', places=[Place(x=0.24, y=0.77, key='c', label='Challenge ctenophora')]),
  'Gallery of Heroes': Map(name='Gallery of Heroes', image='http://online.rapidresizer.com/patternizeimg.php?builtin=logo&id=4cc46cdf56dbdec3f2cde2e1b78f6c5d&bw=1&edge=1&threshold=64&w=', places=[Place(x=0.24, y=0.77, key='n', label=u'RMN'), Place(x=0.06, y=0.46, key='e', label=u'Ehrlichmann'), Place(x=0.27, y=0.42, key='c', label=u'Colson'), Place(x=0.72, y=0.27, key='h', label=u'Haldeman')]),
  'Oval Office': Map(name='Oval Office', image='https://s4.postimg.io/3otw0adgd/ovaloffice.png', places=[Place(x=0.24, y=0.77, key='a', label='War room'), Place(x=0.06, y=0.46, key='v', label="President's phone"), Place(x=0.27, y=0.42, key='i', label='Presidential rocking chair'), Place(x=0.72, y=0.27, key='g', label='Cowboy statue'), Place(x=0.61, y=0.22, key='s', label='Flag of Nixon'), Place(x=0.5, y=0.9, key='d', label='RMN

### parse conditions

In [ ]:
for i, pc in enumerate(p.parsed_contents):
    arr = []
    if isinstance(pc, HarloweMacro):
        if pc.canonical_name == 'if':
            # attach condition to following hook
            

In [128]:
def transform_macro(hm):
    if hm.canonical_name == 'if':
        code = ''
        for c in hm.code:
            if isinstance(c, str):
                code += c
            if isinstance(c, harlowe.HarloweVariable):
                code += 'hv.' + hv.name
        return code
    

### room map

In [13]:
for name, passage in passages.items():
    passage.parse_contents()
    for c in passage.parsed_contents:
        if isinstance(c, harlowe.HarloweLink):
            passage.destinations |= {c.passage_name[0]}
            passages[c.passage_name[0]].parents |= {name}
            
        

In [14]:
for passage in passages.values():
    print passage.name
    print passage.parents,'->', passage.destinations

Gallery of Heroes
set([]) -> set([u'Colson', u'Ehrlichmann', u'RMN', u'Haldeman'])
RMN
set(['Gallery of Heroes']) -> set([])
global
set([]) -> set([])
Ehrlichmann
set(['Gallery of Heroes']) -> set([])
Colson
set(['Gallery of Heroes']) -> set([])
Haldeman
set(['Gallery of Heroes']) -> set([])
Hall of Champions
set([]) -> set([])
